# 02. Image Model Training
Train CNN on satellite imagery

In [1]:
import sys
sys.path.append('../src')

from preprocessing import load_processed_data
from image_model import train_image_model, predict_with_image_model, load_trained_model
from config import *
import torch

train_df, val_df, test_df, final_test_df = load_processed_data()

⚠️  Warning: /Users/yashbansal/Documents/cdc/satellite_property_valuation/notebooks/../data/processed/final_test_processed.csv not found, using test_processed.csv as final_test


## Train Image Model

In [2]:
model, metrics = train_image_model(train_df, val_df)
print(f'Best Val RMSE: ${metrics["val_rmse"]:,.2f}')
print(f'R² Score: {metrics["val_r2"]:.4f}')


TRAINING IMAGE MODEL - TRANSFER LEARNING

⚙️  Configuration:
  Model: efficientnet_b1
  Epochs: 10
  Learning Rate: 0.0003
  Batch Size: 32
  Image Size: 224x224
  Frozen Layers: 80%

🖥️  Device: mps (Apple Silicon GPU)

📊 Log(Price) Normalization:
  Mean:   13.0461
  Std:     0.5262
  Range:  11.2252 -  15.8567

📝 Training Strategy:
  1. Train model to predict log(price)
  2. At inference: y_pred = exp(y_pred_log)
  3. Evaluate on actual price (not log)

📊 Dataset Sizes:
  Training:   12,929 images
  Validation:  3,235 images
  Train batches: 404
  Val batches:   102

🔧 Initializing model...


/Users/yashbansal/Documents/cdc/venv/lib/python3.12/site-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
/Users/yashbansal/Documents/cdc/satellite_property_valuation/notebooks/../src/image_model.py:71: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),



📊 Backbone Parameters:
  Total:        6,513,184
  Frozen:       5,176,784 ( 79.5%)
  Trainable:    1,336,400 ( 20.5%)

📊 Regression Head Parameters: 364,033

📊 Total Trainable: 1,700,433

🚀 Starting training...

Epoch 1/10
--------------------------------------------------------------------------------



📊 Epoch 1 Results:
  Train Loss:     0.81 | Train RMSE: $  347,227.62 | Train R²: 0.1011
  Val Loss:       0.71 | Val RMSE:   $  293,162.15 | Val R²:   0.2413
  Learning Rate: 3.00e-04
  ✅ Best model saved! (Val RMSE: $293,162.15)

Epoch 2/10
--------------------------------------------------------------------------------



📊 Epoch 2 Results:
  Train Loss:     0.74 | Train RMSE: $  335,689.41 | Train R²: 0.1603
  Val Loss:       0.62 | Val RMSE:   $  287,386.97 | Val R²:   0.2709
  Learning Rate: 3.00e-04
  ✅ Best model saved! (Val RMSE: $287,386.97)

Epoch 3/10
--------------------------------------------------------------------------------


...


📊 Epoch 6 Results:
  Train Loss:     0.68 | Train RMSE: $  323,426.05 | Train R²: 0.2205
  Val Loss:       0.58 | Val RMSE:   $  273,287.21 | Val R²:   0.3407
  Learning Rate: 3.00e-04
  ✅ Best model saved! (Val RMSE: $273,287.21)

Epoch 7/10
--------------------------------------------------------------------------------



📊 Epoch 7 Results:
  Train Loss:     0.67 | Train RMSE: $  320,775.05 | Train R²: 0.2332
  Val Loss:       0.55 | Val RMSE:   $  272,022.30 | Val R²:   0.3468
  Learning Rate: 3.00e-04
  ✅ Best model saved! (Val RMSE: $272,022.30)

Epoch 8/10
--------------------------------------------------------------------------------



📊 Epoch 8 Results:
  Train Loss:     0.67 | Train RMSE: $  320,612.71 | Train R²: 0.2340
  Val Loss:       0.55 | Val RMSE:   $  268,763.55 | Val R²:   0.3623
  Learning Rate: 3.00e-04
  ✅ Best model saved! (Val RMSE: $268,763.55)

Epoch 9/10
--------------------------------------------------------------------------------



📊 Epoch 9 Results:
  Train Loss:     0.67 | Train RMSE: $  323,296.33 | Train R²: 0.2211
  Val Loss:       0.55 | Val RMSE:   $  261,623.82 | Val R²:   0.3957
  Learning Rate: 3.00e-04
  ✅ Best model saved! (Val RMSE: $261,623.82)

Epoch 10/10
--------------------------------------------------------------------------------



📊 Epoch 10 Results:
  Train Loss:     0.66 | Train RMSE: $  317,633.52 | Train R²: 0.2482
  Val Loss:       0.54 | Val RMSE:   $  266,729.86 | Val R²:   0.3719
  Learning Rate: 3.00e-04

✅ TRAINING COMPLETED

🏆 Best Results:
  Val RMSE: $  261,623.82
  Val R²:   0.3957

💾 Model saved to: /Users/yashbansal/Documents/cdc/satellite_property_valuation/notebooks/../models/best_image_model.pth
Best Val RMSE: $261,623.82
R² Score: 0.3957


## Generate Predictions

In [3]:
# Load trained model from checkpoint
model, metrics = load_trained_model(MODELS_DIR / 'best_image_model.pth')
print(f'Loaded model - Val RMSE: ${metrics["val_rmse"]:,.2f}, R²: {metrics["val_r2"]:.4f}')

# Generate predictions (only for properties with images)
val_ids, val_preds = predict_with_image_model(model, val_df)

# Create a mapping of IDs to predictions
pred_dict = dict(zip(val_ids, val_preds))

# Add predictions to dataframe (NaN for properties without images)
val_df['image_prediction'] = val_df['id'].map(pred_dict)

# Save results
val_df.to_csv('../data/processed/val_with_image_preds.csv', index=False)
print(f'Validation predictions saved! ({len(val_preds)}/{len(val_df)} properties with images)')


📂 Loading model from: /Users/yashbansal/Documents/cdc/satellite_property_valuation/notebooks/../models/best_image_model.pth

📊 Backbone Parameters:
  Total:        6,513,184
  Frozen:       5,176,784 ( 79.5%)
  Trainable:    1,336,400 ( 20.5%)

📊 Regression Head Parameters: 364,033

📊 Total Trainable: 1,700,433
✅ Model loaded successfully
  Epoch: 8
  Val RMSE: $261,623.82
  Val R²: 0.3957
Loaded model - Val RMSE: $261,623.82, R²: 0.3957

🔮 Generating predictions...
  Loaded normalization: log_mean=13.0461, log_std=0.5262
  Total images: 3,235
  Batches: 102


Predicting: 100%|██████████| 102/102 [00:43<00:00,  2.36it/s]

✅ Generated 3,235 predictions
  Price range: $181,844.58 - $2,555,301.05
Validation predictions saved! (3235/3242 properties with images)


✅ Image model trained!
Next: Run `03_tabular_model_training.ipynb`